In [3]:
# Import packages and check the current directory
import os
import sqlite3
import pandas as pd
print(os.getcwd()) # ensure that you are working in the right directory


C:\Users\thanhkhang\Desktop\Database-for-SusDens\src


In [16]:
# Define a constant for the database name
DATABASE_NAME = "../database/full_database.db"

# Connect to the SQLite database using the constant
conn = sqlite3.connect(DATABASE_NAME)
cursor = conn.cursor()


In [17]:
# Checking the connection
query = 'SELECT * FROM Place'
df_place = pd.read_sql_query(query, conn)

# Display the DataFrame
df_place.head()

,place_id,google_place_id,name,latitude,longitude,description,date,place_category_id,main_type
0,1,ChIJiSgRkqCy6kcR1wQZlcsBHJw,Camping An der Hô,49.700075,5.972239,None,None,7,campground
1,2,ChIJ05bBq-D5v0cRsaZmM833Mnw,EuroParcs Kohnenhof,50.016588,6.135971,None,None,7,campground
2,3,ChIJ4__wcgTiv0cRDQYSE5TQWEw,Camping Janneke,49.853369,6.319640,None,None,7,campground
3,4,ChIJ--4XEvL5v0cRiuKzDFtrg2E,Camping/Cafe Vallée de l'Our,50.004200,6.147010,None,None,7,campground
4,5,ChIJ5XDG2Lbiv0cRfk43fTYj2j8,Camping Bettendorf,49.872944,6.221230,None,None,7,campground


In [14]:


# Create the Place table
cursor.execute('''
CREATE TABLE Place (
    place_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    latitude DECIMAL(9, 6) NOT NULL,
    longitude DECIMAL(9, 6) NOT NULL,
    description TEXT,
    date DATETIME
)
''')

In [17]:

# Alter the Place table to add new columns
cursor.execute('''
ALTER TABLE Place
ADD COLUMN address TEXT
''')

cursor.execute('''
ALTER TABLE Place
ADD COLUMN main_type TEXT
''')

In [ ]:
# Inserting data for Place table

In [23]:

# Define the path to your Excel file
excel_file = "../data/SusDens-Database (Final).xlsx"

# Read the Excel file
try:
    df = pd.read_excel(excel_file, engine='openpyxl')
except FileNotFoundError:
    print(f"File not found: {excel_file}")
    raise

# Prepare the DataFrame for the Place table
df_place = df[['place_id', 'name', 'address', 'geometry.location.lat', 'geometry.location.lng', 'main_type']]
df_place.rename(columns={
    'geometry.location.lat': 'latitude',
    'geometry.location.lng': 'longitude'
}, inplace=True)

# Ensure data types match the schema
df_place['place_id'] = df_place['place_id'].astype(str)
df_place['latitude'] = pd.to_numeric(df_place['latitude'], errors='coerce')
df_place['longitude'] = pd.to_numeric(df_place['longitude'], errors='coerce')
df_place['name'] = df_place['name'].astype(str)
df_place['address'] = df_place['address'].astype(str)
df_place['main_type'] = df_place['main_type'].astype(str)

# Drop rows with NaN values in critical columns
df_place.dropna(subset=['place_id', 'name', 'latitude', 'longitude'], inplace=True)

df_place['latitude'] = df_place['latitude'].astype(float)
df_place['longitude'] = df_place['longitude'].astype(float)

# Function to insert or replace data into the table
def insert_data_from_df(df, table_name, conn):
    # Create a list of tuples from the DataFrame
    data_tuples = [tuple(x) for x in df.to_numpy()]
    
    # Generate SQL command
    placeholders = ', '.join(['?'] * len(df.columns))
    sql = f"INSERT OR REPLACE INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"

    try:
        conn.executemany(sql, data_tuples)
        print(f"Data successfully inserted or replaced in {table_name}")
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Insert or replace data into the Place table
insert_data_from_df(df_place, 'Place', conn)

# Commit changes and close the connection
conn.commit()

Data successfully inserted or replaced in Place


C:\Users\thanhkhang\AppData\Local\Temp\ipykernel_12812\1018018810.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_place.rename(columns={
C:\Users\thanhkhang\AppData\Local\Temp\ipykernel_12812\1018018810.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_place['place_id'] = df_place['place_id'].astype(str)
C:\Users\thanhkhang\AppData\Local\Temp\ipykernel_12812\1018018810.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [ ]:
# For PopularTimes Table

In [71]:

# Drop the existing PopularTimes table
cursor.execute('DROP TABLE IF EXISTS PopularTimes')

# Commit changes
conn.commit()

In [72]:
# Create the table with the correct schema
cursor.execute('''
CREATE TABLE IF NOT EXISTS PopularTimes (
    time_id INTEGER PRIMARY KEY AUTOINCREMENT,
    place_name TEXT NOT NULL,
    day_of_week TEXT NOT NULL,
    Hour_0 INTEGER, Hour_1 INTEGER, Hour_2 INTEGER, Hour_3 INTEGER, Hour_4 INTEGER,
    Hour_5 INTEGER, Hour_6 INTEGER, Hour_7 INTEGER, Hour_8 INTEGER, Hour_9 INTEGER,
    Hour_10 INTEGER, Hour_11 INTEGER, Hour_12 INTEGER, Hour_13 INTEGER, Hour_14 INTEGER,
    Hour_15 INTEGER, Hour_16 INTEGER, Hour_17 INTEGER, Hour_18 INTEGER, Hour_19 INTEGER,
    Hour_20 INTEGER, Hour_21 INTEGER, Hour_22 INTEGER, Hour_23 INTEGER
)
''')

# Commit the changes
conn.commit()

In [73]:
# Query to check table schema
cursor.execute("PRAGMA table_info(PopularTimes)")
schema = cursor.fetchall()

# Print table schema
print("Table Schema:")
for column in schema:
    print(column)


Table Schema:
(0, 'time_id', 'INTEGER', 0, None, 1)
(1, 'place_name', 'TEXT', 1, None, 0)
(2, 'day_of_week', 'TEXT', 1, None, 0)
(3, 'Hour_0', 'INTEGER', 0, None, 0)
(4, 'Hour_1', 'INTEGER', 0, None, 0)
(5, 'Hour_2', 'INTEGER', 0, None, 0)
(6, 'Hour_3', 'INTEGER', 0, None, 0)
(7, 'Hour_4', 'INTEGER', 0, None, 0)
(8, 'Hour_5', 'INTEGER', 0, None, 0)
(9, 'Hour_6', 'INTEGER', 0, None, 0)
(10, 'Hour_7', 'INTEGER', 0, None, 0)
(11, 'Hour_8', 'INTEGER', 0, None, 0)
(12, 'Hour_9', 'INTEGER', 0, None, 0)
(13, 'Hour_10', 'INTEGER', 0, None, 0)
(14, 'Hour_11', 'INTEGER', 0, None, 0)
(15, 'Hour_12', 'INTEGER', 0, None, 0)
(16, 'Hour_13', 'INTEGER', 0, None, 0)
(17, 'Hour_14', 'INTEGER', 0, None, 0)
(18, 'Hour_15', 'INTEGER', 0, None, 0)
(19, 'Hour_16', 'INTEGER', 0, None, 0)
(20, 'Hour_17', 'INTEGER', 0, None, 0)
(21, 'Hour_18', 'INTEGER', 0, None, 0)
(22, 'Hour_19', 'INTEGER', 0, None, 0)
(23, 'Hour_20', 'INTEGER', 0, None, 0)
(24, 'Hour_21', 'INTEGER', 0, None, 0)
(25, 'Hour_22', 'INTEGER', 0, 

In [ ]:
# Inserting data for PopularTimes table

In [76]:
# Define the root folder containing the park folders
root_folder = "../out/popular_times/"

for park_name in os.listdir(root_folder):
    park_folder = os.path.join(root_folder, park_name)
    
    # Check if it is a directory (i.e., a park folder)
    if os.path.isdir(park_folder):
        for file_name in os.listdir(park_folder):
            file_path = os.path.join(park_folder, file_name)
            
            # Check if it is an Excel file
            if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
                # Load the Excel file
                df = pd.read_excel(file_path)
                
                # Insert each row into the database
                for index, row in df.iterrows():
                    cursor.execute('''
                        INSERT INTO PopularTimes (
                            place_name, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4,
                            Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12,
                            Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19,
                            Hour_20, Hour_21, Hour_22, Hour_23
                        ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                    ''', (
                        park_name, row['Day'], row['Hour_0'], row['Hour_1'], row['Hour_2'],
                        row['Hour_3'], row['Hour_4'], row['Hour_5'], row['Hour_6'], row['Hour_7'],
                        row['Hour_8'], row['Hour_9'], row['Hour_10'], row['Hour_11'], row['Hour_12'],
                        row['Hour_13'], row['Hour_14'], row['Hour_15'], row['Hour_16'], row['Hour_17'],
                        row['Hour_18'], row['Hour_19'], row['Hour_20'], row['Hour_21'], row['Hour_22'], row['Hour_23']
                    ))

# Commit changes and close the connection
conn.commit()



In [ ]:
# Creating reviews table version 1 

In [39]:
# Define the path to your Excel file
file_path = "../data/SusDens-Database (Final).xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

In [ ]:
# Creating reviews table ( first edition ) 

In [54]:
# Drop the existing PopularTimes table
cursor.execute('DROP TABLE IF EXISTS reviews')

In [55]:

# Create a table for reviews
cursor.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    place_id INTEGER,
    review TEXT,
    FOREIGN KEY (place_id) REFERENCES places (place_id)
)
''')


In [56]:
# Prepare a list to hold data for insertion
data_to_insert = []

# Process each row in the DataFrame
for index, row in df.iterrows():
    place_id = row['place_id']
    reviews = json.loads(row['reviews'])  # Convert JSON-like string to Python list

    # Ensure reviews is a list
    if isinstance(reviews, list) and reviews:
        for review in reviews:
            # Skip empty reviews
            if review.strip():
                data_to_insert.append((place_id, review.strip()))
    else:
        # If there are no reviews, insert a row with None
        data_to_insert.append((place_id, None))

# Insert data into the database in batches to improve performance
batch_size = 1000
for i in range(0, len(data_to_insert), batch_size):
    batch = data_to_insert[i:i+batch_size]
    cursor.executemany('''
    INSERT INTO reviews (place_id, review)
    VALUES (?, ?)
    ''', batch)
    conn.commit()


In [118]:
# Query to select all data from the Place table
query = 'SELECT * FROM reviews'
df_place = pd.read_sql_query(query, conn)

# Display the DataFrame
df_place.head(100)

,id,place_id,review
0,1,ChIJiSgRkqCy6kcR1wQZlcsBHJw,An excellent family team of the nicest friendl...
1,2,ChIJiSgRkqCy6kcR1wQZlcsBHJw,"Stopped here overnight in a campervan, followi..."
2,3,ChIJiSgRkqCy6kcR1wQZlcsBHJw,"Super friendly owners, Pedro and Nuno. Excelle..."
3,4,ChIJiSgRkqCy6kcR1wQZlcsBHJw,The camping provided a peaceful escape into na...
4,5,ChIJiSgRkqCy6kcR1wQZlcsBHJw,Absolutely lovely palce! The owner was very ni...
...,...,...,...
95,96,ChIJ_xYW7dr9v0cRFnkVqM3a1tU,Came across this one as we drove to our planne...
96,97,ChIJ_xYW7dr9v0cRFnkVqM3a1tU,Really great camping for a good price!\n+ Clea...
97,98,ChIJ_xYW7dr9v0cRFnkVqM3a1tU,We were impressed by the campsite from the out...
98,99,ChIJ_xYW7dr9v0cRFnkVqM3a1tU,We loved this campsite. Lovely cycling routes ...


In [ ]:
# Creating photo reviews 

In [120]:
# Create the photos table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS photos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    file_path TEXT NOT NULL
)
''')

In [ ]:
# Inserting data for phtos table

In [121]:
import zipfile

# Define the base directory for photos
base_directory = r'C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos'  ## please check the data for photos in S:\Echange\Places Data Collection\Nahid\Photos, and then replace with your actual path

# Define supported image file extensions
image_extensions = ('.jpg', '.jpeg', '.png')

def extract_zip_files(directory):
    """
    Extract all zip files in the given directory to a subdirectory.
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.zip'):
                zip_path = os.path.join(root, file)
                extract_path = os.path.splitext(zip_path)[0]  # Extract to the same directory as the zip file
                if not os.path.exists(extract_path):
                    os.makedirs(extract_path)
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_path)
                    print(f"Extracted {zip_path} to {extract_path}")

def process_images(directory):
    """
    Traverse the directory to find image files and add them to the photo records.
    """
    photo_records = []
    for root, dirs, files in os.walk(directory):
        print(f"Scanning directory: {root}")
        for file in files:
            if file.lower().endswith(image_extensions):
                file_path = os.path.join(root, file)
                # Convert absolute path to relative path
                relative_path = os.path.relpath(file_path, base_directory)
                # Extract folder name for 'name'
                folder_name = os.path.basename(os.path.dirname(file_path))
                photo_records.append((folder_name, relative_path))
                print(f"Found image: {relative_path}")

    return photo_records

# Extract zip files
extract_zip_files(base_directory)

# Process images in the base directory
photo_records = process_images(base_directory)

# Check if any images were found
if not photo_records:
    print("No image files found.")
else:
    # Insert data into the photos table
    cursor.executemany('''
    INSERT INTO photos (name, file_path)
    VALUES (?, ?)
    ''', photo_records)

    # Commit the transaction
    conn.commit()
    print(f"Inserted {len(photo_records)} records into the photos table.")

Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Campground.zip to C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Campground
Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Florist.zip to C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Florist
Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Natural Features.zip to C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Natural Features
Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Parks.zip to C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Parks
Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Tourist Attraction.zip to C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Tourist Attraction
Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Zoo.zip to C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Zoo
Extracted C:\Users\thanhkhang\Desktop\Database-for-SusDens\Photos\Parks\Parks\all_places_photos 

In [125]:
# Query to select all data from the Place table
query = 'SELECT * FROM photos'
df_place = pd.read_sql_query(query, conn)

# Display the DataFrame
df_place.head(100)

,id,name,file_path,place_id
0,1,Beter-uit vakantiepark Walsdorf,Campground\Beter-uit vakantiepark Walsdorf\Bet...,ChIJE3h-6Gf8v0cRu2fMozxHpzg
1,2,Beter-uit vakantiepark Walsdorf,Campground\Beter-uit vakantiepark Walsdorf\Bet...,ChIJE3h-6Gf8v0cRu2fMozxHpzg
2,3,Beter-uit vakantiepark Walsdorf,Campground\Beter-uit vakantiepark Walsdorf\Bet...,ChIJE3h-6Gf8v0cRu2fMozxHpzg
3,4,Beter-uit vakantiepark Walsdorf,Campground\Beter-uit vakantiepark Walsdorf\Bet...,ChIJE3h-6Gf8v0cRu2fMozxHpzg
4,5,Beter-uit vakantiepark Walsdorf,Campground\Beter-uit vakantiepark Walsdorf\Bet...,ChIJE3h-6Gf8v0cRu2fMozxHpzg
...,...,...,...,...
95,96,Camping Belle-Vue 2000,Campground\Camping Belle-Vue 2000\Camping Bell...,ChIJdaX7gRZflUcRPrZRRebz2Wc
96,97,Camping Belle-Vue 2000,Campground\Camping Belle-Vue 2000\Camping Bell...,ChIJdaX7gRZflUcRPrZRRebz2Wc
97,98,Camping Belle-Vue 2000,Campground\Camping Belle-Vue 2000\Camping Bell...,ChIJdaX7gRZflUcRPrZRRebz2Wc
98,99,Camping Berkel,Campground\Camping Berkel\Camping Berkel_photo...,ChIJQ87Z1OkAwEcRjjLu2Sc7JEk


In [123]:
# Step 1: Add the new column 'place_id' to the 'photos' table
try:
    cursor.execute('''
    ALTER TABLE photos
    ADD COLUMN place_id TEXT
    ''')
except sqlite3.OperationalError as e:
    print(f"Error adding column: {e}")

# Step 2: Update the 'place_id' in the 'photos' table based on 'places' table
# Assuming 'photos' has a column 'place_name' to match with 'places' table
cursor.execute('''
UPDATE photos
SET place_id = (
    SELECT place_id
    FROM Place
    WHERE Place.name = photos.name
)
''')

# Commit the transaction
conn.commit()

In [124]:
# Verify the changes
cursor.execute('PRAGMA table_info(photos)')
columns = cursor.fetchall()
print("Current columns in 'photos' table:")
for column in columns:
    print(column)

# Optionally check some rows to ensure place_id was updated correctly
cursor.execute('SELECT * FROM photos LIMIT 10')
rows = cursor.fetchall()
print("\nSample rows from 'photos' table:")
for row in rows:
    print(row)


Current columns in 'photos' table:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'file_path', 'TEXT', 1, None, 0)
(3, 'place_id', 'TEXT', 0, None, 0)

Sample rows from 'photos' table:
(1, 'Beter-uit vakantiepark Walsdorf', 'Campground\\Beter-uit vakantiepark Walsdorf\\Beter-uit vakantiepark Walsdorf_photo_0.jpg', 'ChIJE3h-6Gf8v0cRu2fMozxHpzg')
(2, 'Beter-uit vakantiepark Walsdorf', 'Campground\\Beter-uit vakantiepark Walsdorf\\Beter-uit vakantiepark Walsdorf_photo_1.jpg', 'ChIJE3h-6Gf8v0cRu2fMozxHpzg')
(3, 'Beter-uit vakantiepark Walsdorf', 'Campground\\Beter-uit vakantiepark Walsdorf\\Beter-uit vakantiepark Walsdorf_photo_2.jpg', 'ChIJE3h-6Gf8v0cRu2fMozxHpzg')
(4, 'Beter-uit vakantiepark Walsdorf', 'Campground\\Beter-uit vakantiepark Walsdorf\\Beter-uit vakantiepark Walsdorf_photo_3.jpg', 'ChIJE3h-6Gf8v0cRu2fMozxHpzg')
(5, 'Beter-uit vakantiepark Walsdorf', 'Campground\\Beter-uit vakantiepark Walsdorf\\Beter-uit vakantiepark Walsdorf_photo_4.jpg', 'ChIJE3h-6Gf8v

In [126]:
# Step 1: Add the new column 'place_id' to the 'photos' table
try:
    cursor.execute('''
    ALTER TABLE PopularTimes
    ADD COLUMN place_id TEXT
    ''')
except sqlite3.OperationalError as e:
    print(f"Error adding column: {e}")

cursor.execute('''
UPDATE PopularTimes
SET place_id = (
    SELECT place_id
    FROM Place
    WHERE Place.name = PopularTimes.place_name
)
''')

# Commit the transaction
conn.commit()

In [10]:
try:
    # Step 1: Create the new table with the updated schema
    cursor.execute('''
    CREATE TABLE Place_new (
        place_id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id TEXT NOT NULL,
        name TEXT NOT NULL,
        latitude DECIMAL(9, 6) NOT NULL,
        longitude DECIMAL(9, 6) NOT NULL,
        description TEXT,
        date DATETIME
    )
    ''')

    # Step 2: Copy data from the old table to the new table
    cursor.execute('''
    INSERT INTO Place_new (google_place_id, name, latitude, longitude, description, date)
    SELECT place_id, name, latitude, longitude, description, date
    FROM Place
    ''')

    # Step 3: Drop the old table
    cursor.execute('DROP TABLE Place')

    # Step 4: Rename the new table to the original table name
    cursor.execute('ALTER TABLE Place_new RENAME TO Place')

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    # Rollback in case of error
    print(f"An error occurred: {e}")
    conn.rollback()

An error occurred: no such table: Place


In [ ]:
# Create place_category table 

In [ ]:
# Create the place_category table
cursor.execute('''
CREATE TABLE IF NOT EXISTS place_category (
    place_category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    place_category_name TEXT NOT NULL UNIQUE
)
''')

conn.commit()

In [ ]:
# Add place_category_id to Place table
cursor.execute('''
ALTER TABLE Place
ADD COLUMN place_category_id INTEGER
''')

# Make sure to set up the foreign key constraint
cursor.execute('''
PRAGMA foreign_keys = ON;
''')

# Add foreign key constraint to the new column
cursor.execute('''
CREATE TABLE Place_new AS
SELECT * FROM Place
''')

# Drop the old Place table
cursor.execute('DROP TABLE Place')

# Rename the new table to Place
cursor.execute('ALTER TABLE Place_new RENAME TO Place')

# Add foreign key constraint to the Place table
cursor.execute('''
PRAGMA foreign_keys = ON;
''')

conn.commit()

In [ ]:
import pandas as pd
import sqlite3

# Define the path to your Excel file
excel_file = r'C:\Users\thanhkhang\Desktop\Database-for-SusDens\SusDens-Database (Final).xlsx'

# Load the Excel file
df_places = pd.read_excel(excel_file, sheet_name='Sheet1')  # Replace 'Sheet1' with the actual sheet name if different

# Display the first few rows to understand its structure
print(df_places.head())

# Add the new column 'main_type' to the Place table
cursor.execute('''
ALTER TABLE Place
ADD COLUMN main_type TEXT
''')

# Commit the changes
conn.commit()

# Prepare to update the Place table with new data
# Map place_id from the Excel file to google_place_id in the database
for index, row in df_places.iterrows():
    place_id = row['place_id']  # Column name in the Excel file
    main_type = row['main_type']  # Column name for new data in the Excel file
    
    # Update the Place table with the new main_type data
    cursor.execute('''
    UPDATE Place
    SET main_type = ?
    WHERE google_place_id = (
        SELECT google_place_id FROM Place WHERE google_place_id = ?
    )
    ''', (main_type, place_id))

# Commit the changes
conn.commit()

# Optionally, verify the updates
cursor.execute('SELECT * FROM Place LIMIT 10')
rows = cursor.fetchall()
for row in rows:
    print(row)


In [ ]:
# Step 1: Extract unique category names from the Place table
cursor.execute('''
SELECT DISTINCT main_type FROM Place
''')
unique_categories = cursor.fetchall()

# Step 2: Insert unique categories into place_category table
for (category_name,) in unique_categories:
    if category_name:  # Ensure category_name is not None
        cursor.execute('''
        INSERT OR IGNORE INTO place_category (place_category_name)
        VALUES (?)
        ''', (category_name,))

# Commit the changes
conn.commit()

# Create a dictionary to map category names to place_category_id
category_id_map = {}
cursor.execute('SELECT place_category_name, place_category_id FROM place_category')
for row in cursor.fetchall():
    category_id_map[row[0]] = row[1]

# Step 3: Update Place table with the corresponding place_category_id
for category_name, category_id in category_id_map.items():
    cursor.execute('''
    UPDATE Place
    SET place_category_id = ?
    WHERE main_type = ?
    ''', (category_id, category_name))

# Commit the changes
conn.commit()

# Optionally, verify the updates
cursor.execute('SELECT * FROM Place LIMIT 10')
rows = cursor.fetchall()
for row in rows:
    print(row)


In [ ]:
# Modification Part

In [132]:

try:
    # Step 1: Add new place_id columns to the tables
    cursor.execute('ALTER TABLE reviews ADD COLUMN new_place_id INTEGER')
    cursor.execute('ALTER TABLE photos ADD COLUMN new_place_id INTEGER')
    cursor.execute('ALTER TABLE PopularTimes ADD COLUMN new_place_id INTEGER')

    # Step 2: Update the new place_id columns with corresponding values
    cursor.execute('''
    UPDATE reviews
    SET new_place_id = (SELECT place_id FROM Place WHERE Place.google_place_id = reviews.place_id)
    ''')
    
    cursor.execute('''
    UPDATE photos
    SET new_place_id = (SELECT place_id FROM Place WHERE Place.google_place_id = photos.place_id)
    ''')
    
    cursor.execute('''
    UPDATE PopularTimes
    SET new_place_id = (SELECT place_id FROM Place WHERE Place.google_place_id = PopularTimes.place_id)
    ''')

    # Step 3: Drop old columns that are no longer needed
    cursor.execute('ALTER TABLE reviews DROP COLUMN place_id')
    cursor.execute('ALTER TABLE photos DROP COLUMN place_id')
    cursor.execute('ALTER TABLE PopularTimes DROP COLUMN place_name')

    # Step 4: Rename the new columns to the original names
    cursor.execute('ALTER TABLE reviews RENAME COLUMN new_place_id TO place_id')
    cursor.execute('ALTER TABLE photos RENAME COLUMN new_place_id TO place_id')

    # Note: For PopularTimes, we need to re-add the new place_id column as it replaced place_name
    cursor.execute('ALTER TABLE PopularTimes RENAME COLUMN new_place_id TO place_id')

    # Step 5: Add foreign key constraints to the tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS reviews_new (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        review TEXT,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS photos_new (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        name TEXT NOT NULL,
        file_path TEXT NOT NULL,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS PopularTimes_new (
        time_id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        day_of_week TEXT NOT NULL,
        Hour_0 INTEGER, Hour_1 INTEGER, Hour_2 INTEGER, Hour_3 INTEGER, Hour_4 INTEGER,
        Hour_5 INTEGER, Hour_6 INTEGER, Hour_7 INTEGER, Hour_8 INTEGER, Hour_9 INTEGER,
        Hour_10 INTEGER, Hour_11 INTEGER, Hour_12 INTEGER, Hour_13 INTEGER, Hour_14 INTEGER,
        Hour_15 INTEGER, Hour_16 INTEGER, Hour_17 INTEGER, Hour_18 INTEGER, Hour_19 INTEGER,
        Hour_20 INTEGER, Hour_21 INTEGER, Hour_22 INTEGER, Hour_23 INTEGER,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

    # Step 6: Copy data from old tables to new tables
    cursor.execute('INSERT INTO reviews_new (id, place_id, review) SELECT id, place_id, review FROM reviews')
    cursor.execute('INSERT INTO photos_new (id, place_id, name, file_path) SELECT id, place_id, name, file_path FROM photos')
    cursor.execute('INSERT INTO PopularTimes_new (time_id, place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23) SELECT time_id, place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23 FROM PopularTimes')

    # Drop old tables
    cursor.execute('DROP TABLE reviews')
    cursor.execute('DROP TABLE photos')
    cursor.execute('DROP TABLE PopularTimes')

    # Rename new tables to original names
    cursor.execute('ALTER TABLE reviews_new RENAME TO reviews')
    cursor.execute('ALTER TABLE photos_new RENAME TO photos')
    cursor.execute('ALTER TABLE PopularTimes_new RENAME TO PopularTimes')

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    # Rollback in case of error
    print(f"An error occurred: {e}")
    conn.rollback()



An error occurred: error in table reviews after drop column: unknown column "place_id" in foreign key definition


In [34]:
try:
    # Step 1: Create new tables with the updated column name

    # For the photos table
    cursor.execute('''
    CREATE TABLE photos_temp (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id INTEGER,
        name TEXT NOT NULL,
        file_path TEXT NOT NULL,
        FOREIGN KEY (google_place_id) REFERENCES Place (place_id)
    )
    ''')

    # For the reviews table
    cursor.execute('''
    CREATE TABLE reviews_temp (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id INTEGER,
        review TEXT,
        FOREIGN KEY (google_place_id) REFERENCES Place (place_id)
    )
    ''')

    # For the PopularTimes table
    cursor.execute('''
    CREATE TABLE PopularTimes_temp (
        time_id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id INTEGER,
        day_of_week TEXT NOT NULL,
        Hour_0 INTEGER, Hour_1 INTEGER, Hour_2 INTEGER, Hour_3 INTEGER, Hour_4 INTEGER,
        Hour_5 INTEGER, Hour_6 INTEGER, Hour_7 INTEGER, Hour_8 INTEGER, Hour_9 INTEGER,
        Hour_10 INTEGER, Hour_11 INTEGER, Hour_12 INTEGER, Hour_13 INTEGER, Hour_14 INTEGER,
        Hour_15 INTEGER, Hour_16 INTEGER, Hour_17 INTEGER, Hour_18 INTEGER, Hour_19 INTEGER,
        Hour_20 INTEGER, Hour_21 INTEGER, Hour_22 INTEGER, Hour_23 INTEGER,
        FOREIGN KEY (google_place_id) REFERENCES Place (place_id)
    )
    ''')

    # Step 2: Copy data from old tables to new tables

    cursor.execute('''
    INSERT INTO photos_temp (id, google_place_id, name, file_path)
    SELECT id, place_id, name, file_path FROM photos
    ''')

    cursor.execute('''
    INSERT INTO reviews_temp (id, google_place_id, review)
    SELECT id, place_id, review FROM reviews
    ''')

    cursor.execute('''
    INSERT INTO PopularTimes_temp (time_id, google_place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23)
    SELECT time_id, place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23
    FROM PopularTimes
    ''')

    # Step 3: Drop old tables
    cursor.execute('DROP TABLE photos')
    cursor.execute('DROP TABLE reviews')
    cursor.execute('DROP TABLE PopularTimes')

    # Step 4: Rename new tables to original names
    cursor.execute('ALTER TABLE photos_temp RENAME TO photos')
    cursor.execute('ALTER TABLE reviews_temp RENAME TO reviews')
    cursor.execute('ALTER TABLE PopularTimes_temp RENAME TO PopularTimes')

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    # Rollback in case of error
    print(f"An error occurred: {e}")
    conn.rollback()

An error occurred: no such column: name


In [35]:


def table_exists(cursor, table_name):
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
    return cursor.fetchone() is not None


try:
    # Step 1: Drop temporary tables if they exist
    if table_exists(cursor, 'photos_temp'):
        cursor.execute('DROP TABLE photos_temp')
    if table_exists(cursor, 'reviews_temp'):
        cursor.execute('DROP TABLE reviews_temp')
    if table_exists(cursor, 'PopularTimes_temp'):
        cursor.execute('DROP TABLE PopularTimes_temp')

    # Step 2: Create new tables with the updated schema

    # For the photos table
    cursor.execute('''
    CREATE TABLE photos_temp (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id INTEGER,
        file_path TEXT NOT NULL,
        date DATETIME,
        FOREIGN KEY (google_place_id) REFERENCES Place (place_id)
    )
    ''')

    # For the reviews table
    cursor.execute('''
    CREATE TABLE reviews_temp (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id INTEGER,
        review TEXT,
        date DATETIME,
        FOREIGN KEY (google_place_id) REFERENCES Place (place_id)
    )
    ''')

    # For the PopularTimes table
    cursor.execute('''
    CREATE TABLE PopularTimes_temp (
        time_id INTEGER PRIMARY KEY AUTOINCREMENT,
        google_place_id INTEGER,
        day_of_week TEXT NOT NULL,
        Hour_0 INTEGER, Hour_1 INTEGER, Hour_2 INTEGER, Hour_3 INTEGER, Hour_4 INTEGER,
        Hour_5 INTEGER, Hour_6 INTEGER, Hour_7 INTEGER, Hour_8 INTEGER, Hour_9 INTEGER,
        Hour_10 INTEGER, Hour_11 INTEGER, Hour_12 INTEGER, Hour_13 INTEGER, Hour_14 INTEGER,
        Hour_15 INTEGER, Hour_16 INTEGER, Hour_17 INTEGER, Hour_18 INTEGER, Hour_19 INTEGER,
        Hour_20 INTEGER, Hour_21 INTEGER, Hour_22 INTEGER, Hour_23 INTEGER,
        date DATETIME,
        FOREIGN KEY (google_place_id) REFERENCES Place (place_id)
    )
    ''')

    # Step 3: Copy data from old tables to new tables
    cursor.execute('''
    INSERT INTO photos_temp (id, google_place_id, file_path)
    SELECT id, google_place_id, file_path FROM photos
    ''')

    cursor.execute('''
    INSERT INTO reviews_temp (id, google_place_id, review)
    SELECT id, google_place_id, review FROM reviews
    ''')

    cursor.execute('''
    INSERT INTO PopularTimes_temp (time_id, google_place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23)
    SELECT time_id, google_place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23
    FROM PopularTimes
    ''')

    # Step 4: Drop old tables
    cursor.execute('DROP TABLE photos')
    cursor.execute('DROP TABLE reviews')
    cursor.execute('DROP TABLE PopularTimes')

    # Step 5: Rename new tables to original names
    cursor.execute('ALTER TABLE photos_temp RENAME TO photos')
    cursor.execute('ALTER TABLE reviews_temp RENAME TO reviews')
    cursor.execute('ALTER TABLE PopularTimes_temp RENAME TO PopularTimes')

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    # Rollback in case of error
    print(f"An error occurred: {e}")
    conn.rollback()



An error occurred: no such column: google_place_id


In [8]:
import sqlite3

def table_exists(cursor, table_name):
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
    return cursor.fetchone() is not None

def add_place_id_column(cursor):
    # Adding place_id column to existing tables
    if table_exists(cursor, 'photos'):
        cursor.execute('ALTER TABLE photos ADD COLUMN place_id INTEGER')
    if table_exists(cursor, 'reviews'):
        cursor.execute('ALTER TABLE reviews ADD COLUMN place_id INTEGER')
    if table_exists(cursor, 'PopularTimes'):
        cursor.execute('ALTER TABLE PopularTimes ADD COLUMN place_id INTEGER')

def update_place_id_column(cursor):
    # Update place_id column based on google_place_id from Place table
    cursor.execute('''
    UPDATE reviews
    SET place_id = (SELECT place_id FROM Place WHERE Place.google_place_id = reviews.google_place_id)
    ''')

    # Update photos table with place_id; ensure conversion to integer
    cursor.execute('''
    UPDATE photos
    SET place_id = (SELECT place_id FROM Place WHERE Place.google_place_id = CAST(photos.google_place_id AS TEXT))
    ''')

    # Update PopularTimes table with place_id; ensure conversion to integer
    cursor.execute('''
    UPDATE PopularTimes
    SET place_id = (SELECT place_id FROM Place WHERE Place.google_place_id = CAST(PopularTimes.google_place_id AS TEXT))
    ''')

    # Step 1: Add place_id column if it does not exist
    add_place_id_column(cursor)

    # Step 2: Update place_id column based on your mapping logic
    update_place_id_column(cursor)

    # Step 3: Drop temporary tables if they exist
    if table_exists(cursor, 'photos_temp'):
        cursor.execute('DROP TABLE photos_temp')
    if table_exists(cursor, 'reviews_temp'):
        cursor.execute('DROP TABLE reviews_temp')
    if table_exists(cursor, 'PopularTimes_temp'):
        cursor.execute('DROP TABLE PopularTimes_temp')

    # Step 4: Create new tables with place_id as the foreign key
    cursor.execute('''
    CREATE TABLE photos_temp (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        file_path TEXT NOT NULL,
        date DATETIME,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

    cursor.execute('''
    CREATE TABLE reviews_temp (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        review TEXT,
        date DATETIME,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

    cursor.execute('''
    CREATE TABLE PopularTimes_temp (
        time_id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        day_of_week TEXT NOT NULL,
        Hour_0 INTEGER, Hour_1 INTEGER, Hour_2 INTEGER, Hour_3 INTEGER, Hour_4 INTEGER,
        Hour_5 INTEGER, Hour_6 INTEGER, Hour_7 INTEGER, Hour_8 INTEGER, Hour_9 INTEGER,
        Hour_10 INTEGER, Hour_11 INTEGER, Hour_12 INTEGER, Hour_13 INTEGER, Hour_14 INTEGER,
        Hour_15 INTEGER, Hour_16 INTEGER, Hour_17 INTEGER, Hour_18 INTEGER, Hour_19 INTEGER,
        Hour_20 INTEGER, Hour_21 INTEGER, Hour_22 INTEGER, Hour_23 INTEGER,
        date DATETIME,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

    # Step 5: Copy data from old tables to new tables
    cursor.execute('''
    INSERT INTO photos_temp (id, place_id, file_path, date)
    SELECT id, place_id, file_path, date FROM photos
    ''')

    cursor.execute('''
    INSERT INTO reviews_temp (id, place_id, review, date)
    SELECT id, place_id, review, date FROM reviews
    ''')

    cursor.execute('''
    INSERT INTO PopularTimes_temp (time_id, place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23, date)
    SELECT time_id, place_id, day_of_week, Hour_0, Hour_1, Hour_2, Hour_3, Hour_4, Hour_5, Hour_6, Hour_7, Hour_8, Hour_9, Hour_10, Hour_11, Hour_12, Hour_13, Hour_14, Hour_15, Hour_16, Hour_17, Hour_18, Hour_19, Hour_20, Hour_21, Hour_22, Hour_23, date
    FROM PopularTimes
    ''')

    # Step 6: Drop old tables
    cursor.execute('DROP TABLE photos')
    cursor.execute('DROP TABLE reviews')
    cursor.execute('DROP TABLE PopularTimes')

    # Step 7: Rename new tables to original names
    cursor.execute('ALTER TABLE photos_temp RENAME TO photos')
    cursor.execute('ALTER TABLE reviews_temp RENAME TO reviews')
    cursor.execute('ALTER TABLE PopularTimes_temp RENAME TO PopularTimes')

    # Commit the changes
    conn.commit()

except sqlite3.Error as e:
    # Rollback in case of error
    print(f"An error occurred: {e}")
    conn.rollback()

finally:
    # Ensure the connection is closed
    if conn:
        conn.close()


An error occurred: duplicate column name: place_id


In [ ]:
# Inserting data for reviews table

In [16]:
import sqlite3
import zipfile
import os
import re

def create_reviews_table(cursor):
    """Create the reviews table if it does not exist."""
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS reviews (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        place_id INTEGER,
        author TEXT,
        rating INTEGER,
        date DATETIME,
        review TEXT,
        FOREIGN KEY (place_id) REFERENCES Place (place_id)
    )
    ''')

def clear_reviews_table(cursor):
    """Delete all records from the reviews table."""
    cursor.execute('DELETE FROM reviews')

def add_new_columns(cursor):
    """Add new columns to the reviews table if they do not already exist."""
    try:
        cursor.execute('ALTER TABLE reviews ADD COLUMN author TEXT')
    except sqlite3.OperationalError:
        pass  # Column already exists
    
    try:
        cursor.execute('ALTER TABLE reviews ADD COLUMN rating INTEGER')
    except sqlite3.OperationalError:
        pass  # Column already exists
    
    try:
        cursor.execute('ALTER TABLE reviews ADD COLUMN date DATETIME')
    except sqlite3.OperationalError:
        pass  # Column already exists
    
    try:
        cursor.execute('ALTER TABLE reviews ADD COLUMN review TEXT')
    except sqlite3.OperationalError:
        pass  # Column already exists

def extract_zip_files(zip_dir, extract_to_dir):
    """Extract all ZIP files from the given directory to a specified directory."""
    for zip_file_name in os.listdir(zip_dir):
        zip_file_path = os.path.join(zip_dir, zip_file_name)
        if zip_file_name.endswith('.zip'):
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to_dir)

def process_reviews_files(cursor, base_dir):
    """Process each folder and file to extract and insert review data into the database."""
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            # Get place_id from the database
            cursor.execute('SELECT place_id FROM Place WHERE name = ?', (folder_name,))
            result = cursor.fetchone()
            if result:
                place_id = result[0]
                for file_name in os.listdir(folder_path):
                    file_path = os.path.join(folder_path, file_name)
                    if file_name.endswith('.txt'):
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                            # Extract properties
                            author = re.search(r'Author:\s*(.+)', content)
                            rating = re.search(r'Rating:\s*(\d+)', content)
                            date = re.search(r'Date:\s*(\S+ \S+)', content)
                            review = re.search(r'Review:\s*(.+)', content, re.DOTALL)
                            
                            author = author.group(1) if author else None
                            rating = int(rating.group(1)) if rating else None
                            date = date.group(1) if date else None
                            review = review.group(1).strip() if review else None
                            
                            # Insert into database
                            cursor.execute('''
                            INSERT INTO reviews (place_id, author, rating, date, review)
                            VALUES (?, ?, ?, ?, ?)
                            ''', (place_id, author, rating, date, review))

def main():
    try:
        # Define paths
        db_path = "../database/full_database.db"
        zip_dir = r'C:\Users\thanhkhang\Desktop\Database-for-SusDens\Updated Reviews' # Please check the data for updated reviews in S:\Echange\Places Data Collection\Kay\Database-for-SusDens\Updated Reviews
        extract_to_dir = r'C:\Users\thanhkhang\Desktop\Database-for-SusDens\Updated Reviews\extracted' # Replace by your working path 
        
        # Create the extraction directory if it does not exist
        os.makedirs(extract_to_dir, exist_ok=True)
        
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Step 1: Create the reviews table if it doesn't exist
        create_reviews_table(cursor)
        
        # Step 2: Clear existing reviews
        clear_reviews_table(cursor)
        
        # Step 3: Add new columns
        add_new_columns(cursor)
        
        # Step 4: Extract ZIP files
        extract_zip_files(zip_dir, extract_to_dir)
        
        # Step 5: Process files and insert into database
        process_reviews_files(cursor, extract_to_dir)
        
        # Commit the changes
        conn.commit()
    
    except sqlite3.Error as e:
        # Rollback in case of error
        print(f"An error occurred: {e}")
        conn.rollback()
    
    finally:
        # Ensure the connection is closed
        if conn:
            conn.close()

if __name__ == "__main__":
    main()


In [19]:

# Step 1: Set reviews with no text (empty) but with a rating to NULL
cursor.execute('''
    UPDATE reviews
    SET review = NULL
    WHERE (review IS NULL OR TRIM(review) = '') AND rating IS NOT NULL
''')

# Step 2: Delete reviews with no text and no rating
cursor.execute('''
    DELETE FROM reviews
    WHERE (review IS NULL OR TRIM(review) = '') AND (rating IS NULL OR rating = '')
''')

# Commit the changes
conn.commit()